In [ ]:
# FASTTEXT AND RCNN DATA PREPARATION
MAX_NAME_SEQ = 40
MAX_TEXT = len(vocabulary_one)+ 1

MAX_age    = np.max(df_train['age_cat'].max()) + 1
MAX_sex   = np.max(df_train['sex'].max()) + 1
MAX_STAY   = np.max(df_train['stay_cat'].max()) + 1
MAX_LANG   = np.max(df_train['lang'].max()) + 1
MAX_ER       = np.max(df_train['er'].max()) + 1

def preprocess_keras(text):
    return [ vocabulary_one[w] for w in (text.split())[:MAX_ITEM_DESC_SEQ] ]
def preprocess_keras_df(df):
    return df.apply( preprocess_keras )

start_time = time.time()
df_train['comment'] = parallelize_dataframe(df_train['name'], preprocess_keras_df)

def get_keras_fasttext(df):
    X = {
        'comment': pad_sequences(df['comment'], maxlen=MAX_NAME_SEQ),
        
        'age': np.array(df['age_cat']),
        'sex': np.array(df['sex']),
        'stay': np.array(df['stay_cat']),
        'lang': np.array(df['lang']),
        'er': np.array(df['er']),
        
    }
    return X
train_keras      = get_keras_fasttext(df_train)
print('[{}] Finished Converting to Sequence for FASTTEXT NN'.format(time.time() - start_time))

#FASTTEXT MODEL
def fasttext_model():
    name = Input(shape=[train_keras["name"].shape[1]], name="name")
    item_desc = Input(shape=[train_keras["item_desc"].shape[1]], name="item_desc")
    brand_name = Input(shape=[1], name="brand_name")
    category_name = Input(shape=[1], name="category_name")
    category1 = Input(shape=[1], name="category1")
    category2 = Input(shape=[1], name="category2")
    category3 = Input(shape=[1], name="category3")
    item_condition = Input(shape=[1], name="item_condition")
    shipping = Input(shape=[1], name="shipping")
    temp = Input(shape=[1], name="temp")
    temp2 = Input(shape=[1], name="temp2")
    
    shared_embedding = Embedding(MAX_TEXT, 50)    
    emb_name = shared_embedding (name)
    emb_item_desc = shared_embedding (item_desc)
    
    val=10
    emb_brand_name = Flatten() ( Embedding(MAX_BRAND, val)(brand_name)    )
    emb_category1 = Flatten() ( Embedding(MAX_CATEGORY, val)(category1) )
    emb_category2 = Flatten() ( Embedding(MAX_CATEGORY, val)(category2) )
    emb_category3 = Flatten() ( Embedding(MAX_CATEGORY, val)(category3) )
    
    emb_name = GlobalAveragePooling1D( name='output_name_max' )(emb_name)
    emb_item_desc = GlobalAveragePooling1D(name='output_item_max' )(emb_item_desc)

    x = concatenate([  item_condition , shipping, emb_name, emb_item_desc, 
    emb_brand_name, emb_category1, emb_category2, emb_category3    ])
    x = BatchNormalization()(x)
    x = Dense(1024)(x)
    x = Activation('relu')(x)

    x = Dense(1, activation="linear") (x)
    model = Model([name, item_desc,  brand_name , category_name,
                   category1, category2, category3, item_condition, shipping, temp, temp2],
                   x)
    optimizer = Adam(.002)
    model.compile(loss="mse", optimizer=optimizer)

    return model

BATCH_SIZE = 128
epochs = 4

fasttext_model = fasttext_model()

print("Fitting FASTTEXT NN model ...")

for ep in range(epochs):
    BATCH_SIZE = int(BATCH_SIZE*2)
    fasttext_model.fit(  train_keras, (df_train.price.values-mean_price), 
                      batch_size=BATCH_SIZE, epochs=1, verbose=10 )
del train_keras
gc.collect()
